In [35]:
import numpy as np
import pandas as pd
import geopandas as gpd

import geopy
from geopy.geocoders import Nominatim

In [36]:
df1 = pd.read_excel(r'..\data\raw\2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'Rep Addresses');
# df2 = pd.read_excel(r'C:\Users\filename.xlsx', sheet_name= 'Sheet2');

In [37]:
df1.head(5)

,Division,Region,Region\nName,Territory,Name,Last Name,* FAX # \nCELL #,Address_1,City,State,Zip,E-mail,Country,Comments,Regionals to Sort Separate,Notes
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN


In [38]:
df1.shape

(112, 16)

In [34]:
df1.columns = 

SyntaxError: invalid syntax (4169553798.py, line 1)

In [39]:
df1 = df1.rename(columns={'Division': 'division',
                          'Region': 'region_code',
                          'Territory' : 'territory',
                          'Name': 'name',
                          'Last Name' : 'last_name',
                          'Region\nName': 'region_name', 
                          ' * FAX # \nCELL #': 'fax/cell',
                         'Address_1' : 'street_address',
                          'City' : 'city',
                          'State' : 'state',
                          'Zip' : 'zip',
                          'E-mail' : 'email',
                          'Country' : 'country',
                          'Comments' : 'comments',
                          'Regionals to Sort Separate' : 'regionals_to_sort_separate',
                          'Notes' : 'notes'
                         })


In [40]:
df1.columns

Index(['division', 'region_code', 'region_name', 'territory', 'name',
       'last_name', 'fax/cell', 'street_address', 'city', 'state', 'zip',
       'email', 'country', 'comments', 'regionals_to_sort_separate', 'notes'],
      dtype='object')

In [41]:
df1

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,regionals_to_sort_separate,notes
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df1 = df1.dropna(how='all')


In [43]:
df1['full_address'] = df1['street_address'].str.strip() + ', ' + df1['city'].str.strip() + ', ' + df1['state'].str.strip()

C:\Users\valesa10\AppData\Local\Temp\ipykernel_6776\974523268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['full_address'] = df1['street_address'].str.strip() + ', ' + df1['city'].str.strip() + ', ' + df1['state'].str.strip()


In [44]:
df1

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,regionals_to_sort_separate,notes,full_address
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN,"1610 Jefferson St., Kerrville, TX"
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN,"3930 Bethlehem Rd., Boones Mill, VA"
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN,"33475 Berns Shore Dr., Adel, IA"
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN,"1218 Fairfield Estates Lane, Crownsville, MD"
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN,"5225 N. New England Ave., Chicago, IL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,A,B5,LA/Portland,HC,Kevin,Yebra,916.417.8883,233 Perretti Ct.,Roseville,CA,95747,kevin.yebra@batesville.com,NaN,NaN,NaN,NaN,"233 Perretti Ct., Roseville, CA"
103,NaN,NaN,NaN,NaN,NaN,In Training,Regional Directors,Consultant-calling on a \nterritory but NOT a rep,SR Sales Rep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,Special Mailing Instructions,SSR\nSales Support Reps,NO MAIL AT PRESENT TIME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,International,Strategic Accts Specialist,GM - Growth Market Reps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Geocoding

def geocode(address):
    geolocator = Nominatim(user_agent="geolocating")
    location = geolocator.geocode(address)
#     print(location.raw)
    return str(location.latitude)
    
df2 =df1.head(5)

In [85]:
df2['lat_long'] = df2['full_address'].apply(geocode)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [68]:
df2

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,regionals_to_sort_separate,notes,full_address
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN,"1610 Jefferson St., Kerrville, TX"
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN,"3930 Bethlehem Rd., Boones Mill, VA"
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN,"33475 Berns Shore Dr., Adel, IA"
3,A,H3,Allentown,R5,Brian,Ager,443-858-7970,1218 Fairfield Estates Lane,Crownsville,MD,21032,brian.ager@batesville.com,NaN,NaN,NaN,NaN,"1218 Fairfield Estates Lane, Crownsville, MD"
4,A,H4,Batesville,BB,Jason,Aldrin,* 773.631.9042\n773.505.0926,5225 N. New England Ave.,Chicago,IL,60656,jason.aldrin@batesville.com,NaN,NaN,NaN,NaN,"5225 N. New England Ave., Chicago, IL"
